<a href="https://colab.research.google.com/github/AROMAL2059/-UEL-Machine-Learning-with-big-data/blob/main/Week_6%3DRecSys_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!git add .
!git commit -m "Initial commit"
!git branch -M main  # If needed, rename the branch to 'main'
!git push -u origin main  # Push to the 'main' branch



error: '-UEL-Machine-Learning-with-big-data/' does not have a commit checked out
fatal: adding files failed
On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	-UEL-Machine-Learning-with-big-data/
	.config/
	movies.csv
	ratings.csv
	sample_data/
	tags.csv

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/AROMAL2059/-UEL-Machine-Learning-with-big-data.git'


In [42]:
!git config --global user.email "aromal2059@gmail.com"
!git config --global user.name "AROMAL2059"


In [14]:
#################################################################################
# In this program, we shall be developing a movie Rec System
#################################################################################


!pip install pyspark


In [15]:
import os

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Rec System") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [16]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [17]:
movies = spark.read.csv("/content/movies.csv",inferSchema=True,header=True)
ratings = spark.read.csv("/content/ratings.csv",inferSchema=True,header=True)

In [18]:
ratings.show()
movies.show()


+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [19]:

ratings.printSchema()
movies.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [22]:
from pyspark.sql import functions as F
#The first code snippet not only drops the 'timestamp'
# column but also performs additional operations such as casting certain columns
# to specific data types (userId to integer, movieId to integer, and rating to float) using the withColumn method.
ratings = ratings.\
    withColumn('userId', F.col('userId').cast('integer')).\
    withColumn('movieId', F.col('movieId').cast('integer')).\
    withColumn('rating', F.col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



In [23]:
# Sparsity shows how sparse is the matrix, e.g. 99% sparsity shows that, 99% of the entries in the matrix are missing.
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


In [24]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [25]:

# Group data by movieId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|     50|  204|
|   2858|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



In [ ]:
##################################
# ALS MODEL --- Alternative Least Square Method

In [26]:

# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [27]:
# Split the ratings dataframe into training and test data
(train, test) = ratings.randomSplit([0.8, 0.2], seed=42)

# Set the ALS hyperparameters
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    rank =10,
    maxIter =10,
    regParam =.1,
    coldStartStrategy="drop",
    nonnegative =True,
    implicitPrefs = False
)

# Fit the model to the training_data
als_model = als.fit(train)

# Generate predictions on the test_data
test_predictions = als_model.transform(test)
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|   4896|   4.0| 3.6148007|
|   148|   5618|   3.0| 3.5272582|
|   148|   7153|   3.0| 3.3923879|
|   148|  40629|   5.0| 3.3700583|
|   148|  40815|   4.0|  3.649026|
|   148|  60069|   4.5| 3.6771636|
|   148|  68954|   4.0| 3.7244067|
|   148|  69844|   4.0| 3.6883917|
|   148|  79132|   1.5| 3.4684443|
|   148|  79702|   4.0| 3.3601599|
|   148|  81834|   4.0|  3.668439|
|   148|  81847|   4.5| 3.2145953|
|   148|  98243|   4.5| 3.5504165|
|   148|  98491|   5.0| 3.7964327|
|   148| 108932|   4.0| 3.5365646|
|   463|   1088|   3.5| 3.6466417|
|   463|   1221|   4.5| 4.0618305|
|   463|   2028|   4.5| 4.1662807|
|   463|   2167|   3.0| 3.5731964|
|   463|   3448|   3.0| 3.7726212|
+------+-------+------+----------+
only showing top 20 rows



In [28]:
# Import RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

# Complete the evaluator code
evaluator = RegressionEvaluator(metricName="rmse", labelCol="userId", predictionCol="rating")

# Extract the 3 parameters
print(evaluator.getMetricName())
print(evaluator.getLabelCol())
print(evaluator.getPredictionCol())


# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print ('RMSE = ', RMSE)

rmse
userId
rating
RMSE =  368.0588125025974


In [29]:
# Generate n Recommendations for all users
nrecommendations = als_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{96004, 5.859417...|
|     2|[{183897, 4.95190...|
|     3|[{74754, 5.534515...|
|     4|[{7579, 4.9566784...|
|     5|[{25825, 5.038783...|
|     6|[{171, 5.057845},...|
|     7|[{1658, 4.946783}...|
|     8|[{25771, 5.219259...|
|     9|[{25771, 5.381905...|
|    10|[{1172, 4.7881317...|
+------+--------------------+



In [30]:
#Explaining the recommendations

from pyspark.sql.functions import split, explode

nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', F.col("rec_exp.movieId"), F.col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|  96004|5.8594174|
|     1|   3379|5.8594174|
|     1| 104875|5.7598352|
|     1|  33649| 5.717746|
|     1|  93988|5.6721992|
|     1|   8235| 5.642348|
|     1|   6201| 5.642348|
|     1|  25771| 5.619649|
|     1| 171495| 5.599961|
|     1| 177593|5.5618978|
+------+-------+---------+



/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `git clone <https://github.com/AROMAL2059/-UEL-Machine-Learning-with-big-data.git>'


In [31]:
#Explaining the recommendations
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()


+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|  93008|   100| 4.997987|Very Potter Seque...|      Comedy|Musical|
|  77846|   100| 4.997987| 12 Angry Men (1997)|         Crime|Drama|
|  25906|   100| 4.997987|Mr. Skeffington (...|       Drama|Romance|
|  45503|   100|4.9651494|Peaceful Warrior ...|               Drama|
|  74282|   100| 4.949336|Anne of Green Gab...|Children|Drama|Ro...|
| 158872|   100| 4.948445|Sausage Party (2016)|    Animation|Comedy|
| 104875|   100| 4.911713|History of Future...|Adventure|Comedy|...|
|  93988|   100|  4.88517|North & South (2004)|       Drama|Romance|
|  98154|   100|  4.87809|      Lincoln (2012)|           Drama|War|
|  25771|   100|4.8726535|Andalusian Dog, A...|             Fantasy|
+-------+------+---------+--------------------+--------------------+



In [32]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()


+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+



In [39]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
